#Кинематика движения в соответствии с разметкой json

In [ ]:
# Подключение Google Диска
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Исходяшее видео
video_1 = '/content/drive/My Drive/video/cam1.mp4'
video_2 = '/content/drive/My Drive/video/cam2.mp4'
video_3 = '/content/drive/My Drive/video/cam3.mp4'
video_4 = '/content/drive/My Drive/video/cam4.mp4'

video_list =[video_1, video_2, video_3, video_4]

In [ ]:
import subprocess
import json
import os
from moviepy.editor import VideoFileClip

# Функция для получения информации о кодеке с помощью ffprobe
def get_video_codec(video_path):
    command = [
        'ffprobe',
        '-v', 'error',
        '-select_streams', 'v:0',
        '-show_entries', 'stream=codec_name',
        '-of', 'json',
        video_path
    ]
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    if result.returncode != 0:
        print(f"Error getting codec for {video_path}: {result.stderr.decode()}")
        return 'Unknown'

    info = json.loads(result.stdout)
    codec = info['streams'][0]['codec_name'] if 'streams' in info and len(info['streams']) > 0 else 'Unknown'
    return codec

# Функция для вывода информации о видео
def print_video_info(video_paths):
    for video_path in video_paths:
        if not os.path.exists(video_path):
            print(f"File not found: {video_path}")
            continue

        try:
            with VideoFileClip(video_path) as video:
                codec = get_video_codec(video_path)
                duration = video.duration
                fps = video.fps
                frame_count = int(fps * duration)  # Количество кадров

                print(f"Video: {video_path}")
                print(f"  Duration: {duration:.2f} seconds")
                print(f"  FPS: {fps:.2f}")
                print(f"  Frame Count: {frame_count}")
                print(f"  Resolution: {video.size[0]}x{video.size[1]}")
                print(f"  Codec: {codec}")
                print()
        except Exception as e:
            print(f"Error processing {video_path}: {e}")

In [ ]:
import cv2
import os

# Функция для получения информации о видео
def print_video_info(video_paths):
    for video_path in video_paths:
        if not os.path.exists(video_path):
            print(f"File not found: {video_path}")
            continue

        try:
            # Открываем видеофайл с помощью OpenCV
            video = cv2.VideoCapture(video_path)

            if not video.isOpened():
                print(f"Error opening video file: {video_path}")
                continue

            # Получаем информацию о видео
            fps = video.get(cv2.CAP_PROP_FPS)
            frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
            duration = frame_count / fps if fps > 0 else 0
            width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
            height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

            # Закрываем видеофайл
            video.release()

            print(f"Video: {video_path}")
            print(f"  Duration: {duration:.2f} seconds")
            print(f"  FPS: {fps:.2f}")
            print(f"  Frame Count: {frame_count}")
            print(f"  Resolution: {width}x{height}")
            #print(f"  Codec: Unknown (OpenCV does not provide codec information)")
            print()
        except Exception as e:
            print(f"Error processing {video_path}: {e}")


In [ ]:
# Вызов функции
print_video_info(video_list)

Video: /content/drive/My Drive/video/cam1.mp4
  Duration: 818.00 seconds
  FPS: 29.79
  Frame Count: 24369
  Resolution: 1920x1080

Video: /content/drive/My Drive/video/cam2.mp4
  Duration: 1127.58 seconds
  FPS: 30.10
  Frame Count: 33945
  Resolution: 1920x1080

Video: /content/drive/My Drive/video/cam3.mp4
  Duration: 876.90 seconds
  FPS: 29.97
  Frame Count: 26279
  Resolution: 1920x1080

Video: /content/drive/My Drive/video/cam4.mp4
  Duration: 802.27 seconds
  FPS: 30.00
  Frame Count: 24068
  Resolution: 2224x1080



In [ ]:
#@title Сохранение кадра видео
import cv2 as cv
import os

# Путь к видео
video_path = '/content/drive/My Drive/video/cam3.mp4'

# Номер кадра, который нужно сохранить
frame_number = int(input('Введите номер кадра', ))

# Путь к папке для сохранения изображений на Google Диске
image_path = "/content/drive/My Drive/video/frame/"

# Создаем папку, если она не существует
os.makedirs(image_path, exist_ok=True)

# Открываем видео
cap = cv.VideoCapture(video_path)

# Устанавливаем указатель на нужный кадр
cap.set(cv.CAP_PROP_POS_FRAMES, frame_number)

# Читаем кадр
ret, frame = cap.read()

# Проверяем, был ли кадр успешно прочитан
if ret:
    # Сохраняем кадр
    cv.imwrite(os.path.join(image_path, f'frame_{frame_number}.png'), frame)
    print (f'кадр frame_{frame_number}.png сохранен')
else:
    print(f"Не удалось прочитать кадр {frame_number} из видео.")

# Освобождаем ресурсы
cap.release()


Введите номер кадра6698
кадр frame_6698.png сохранен


In [ ]:
#@title Чтение json и сохранение в датафрейм

import pandas as pd
import json

# Считываем файл cam1.json
with open('cam3.json', 'r') as file:
    data = json.load(file)

# Создаем список для хранения преобразованных данных
rows = []

# Проходим по каждому элементу в загруженных данных
for video in data:
    for label in video['videoLabels']:
        for range_ in label['ranges']:
            rows.append({
                'start': range_['start'],
                'end': range_['end'],
                'timelinelabels': label['timelinelabels'][0]  # Предполагаем, что timelinelabels содержит только один элемент
            })

# Преобразуем список в DataFrame
df = pd.DataFrame(rows)

In [ ]:
df

,start,end,timelinelabels
0,3531,3557,L-Turn-90
1,3531,3588,L-Kokutsu-Dachi
2,3531,3557,L-Jodan-Haiwan-Uke
3,3558,3567,R-Chudan-Tettsui
4,3568,3588,L-Chudan-Yoko-Tettsui
...,...,...,...
983,23413,23472,L-Jodan-Age-Uke
984,23474,23517,L-Turn-135
985,23518,23551,Kamae
986,23552,23588,Rei


In [ ]:
df.to_excel('cam3.xlsx')

In [ ]:
#@title Обрезка видео

import os

# Путь к видео
video_paths = {
    'cam1.mp4': '/content/drive/My Drive/video/cam1.mp4',
    'cam2.mp4': '/content/drive/My Drive/video/cam2.mp4',
    'cam3.mp4': '/content/drive/My Drive/video/cam3.mp4',
    'cam4.mp4': '/content/drive/My Drive/video/cam4.mp4'
}

# Временные метки для обрезки (начало и конец в секундах)
cut_times = {
    'cam1.mp4': (180, 220),
    'cam2.mp4': (495, 535),
    'cam3.mp4': (213, 253),
    'cam4.mp4': (152, 192)
}

# Папка для сохранения обрезанных видео
output_folder = '/content/drive/My Drive/video/cell_1/'

# Создаем папку для обрезанных видео, если она не существует
os.makedirs(output_folder, exist_ok=True)

# Обрезка видео
for video_name, (start, end) in cut_times.items():
    video_path = video_paths[video_name]
    output_video_path = os.path.join(output_folder, f'cut_{video_name}')

    # Вычисляем длительность обрезки
    duration = end - start

    # Команда для обрезки видео
    command = f"ffmpeg -i '{video_path}' -ss {start} -t {duration} -c copy '{output_video_path}'"

    # Выполняем команду
    os.system(command)

print("Обрезка видео завершена.")

Обрезка видео завершена.


In [ ]:
#@title Создание клипа с 4-мя распознанными видео - участник 1

'''
Synchronizing...
--> Camera cam03 and cam01: -29 frames offset, correlation 0.48.
--> Camera cam03 and cam02: 32 frames offset, correlation 0.38.
--> Camera cam03 and cam04: 38 frames offset, correlation 0.31.
'''

import subprocess
import json
import cv2 as cv
from moviepy.editor import VideoFileClip, clips_array

# Исходные видео
video_output_1 = '/content/drive/My Drive/video/cell_1/cam01_pose.mp4'
video_output_2 = '/content/drive/My Drive/video/cell_1/cam02_pose.mp4'
video_output_3 = '/content/drive/My Drive/video/cell_1/cam03_pose.mp4'
video_output_4 = '/content/drive/My Drive/video/cell_1/cam04_pose.mp4'

output_video_list = [video_output_1, video_output_2, video_output_3, video_output_4]

# Получение FPS для каждого видео
fps_list = []
for video_path in output_video_list:
    cap = cv.VideoCapture(video_path)
    fps = cap.get(cv.CAP_PROP_FPS)
    print(fps)
    fps_list.append(fps)
    cap.release()

# Загрузка видео
video_clips = [VideoFileClip(video) for video in output_video_list]

# Синхронизация видео
# cam03 (основное видео)
cam03_duration = video_clips[2].duration

# cam01: опережает на -29 кадров
cam01_offset = -29 / fps_list[0]  # 35 кадров
video_clips[0] = video_clips[0].subclip(max(0, cam01_offset), cam03_duration)

# cam02: отстает на 32 кадров
cam02_offset = 32 / fps_list[1]  # 30 кадров
video_clips[1] = video_clips[1].subclip(max(0, cam02_offset), cam03_duration)

# cam04: отстает на 38 кадров
cam04_offset = 38 / fps_list[3]  # -8 кадров
video_clips[3] = video_clips[3].subclip(max(0, cam04_offset), cam03_duration)

# Создание плитки из видео
final_video = clips_array([[video_clips[0], video_clips[1]],
                            [video_clips[2], video_clips[3]]])

# Сохранение и вывод финального видео
final_video_path = '/content/drive/My Drive/video/cell_1/final_video_01.mp4'
final_video.write_videofile(final_video_path, codec='libx264')

  if event.key is 'enter':



29.8
30.186
30.002
30.0
Moviepy - Building video /content/drive/My Drive/video/cell_1/final_video_01.mp4.
Moviepy - Writing video /content/drive/My Drive/video/cell_1/final_video_01.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/My Drive/video/cell_1/final_video_01.mp4


# Файл кинематики участника 1 после Pose2Sim

In [ ]:
import zipfile
import os

file_path = '/content/drive/MyDrive/Colab Notebooks/AI_University/Стажировки/!03-Каратэ/10_weak/pose-sync_01.zip'

# Папка, куда вы хотите разархивировать файлы
extract_folder = '/content/drive/MyDrive/Colab Notebooks/AI_University/Стажировки/!03-Каратэ/10_weak/kinematics_01/'

# Создаем папку, если она не существует
os.makedirs(extract_folder, exist_ok=True)

# Разархивируем файл
with zipfile.ZipFile(file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

print(f'Файлы разархивированы в {extract_folder}')

Файлы разархивированы в /content/drive/MyDrive/Colab Notebooks/AI_University/Стажировки/!03-Каратэ/10_weak/kinematics_01/


In [ ]:
#@title Загружаем файл mot
import pandas as pd

def read_mot_file(file_path):
    with open(file_path, 'r') as file:
        # Читаем все строки файла
        lines = file.readlines()

    # Ищем строку с заголовками (первая строка данных)
    for i, line in enumerate(lines):
        if line.startswith('time'):
            header_index = i
            break
    else:
        raise ValueError("Header not found in the file.")

    # Извлекаем названия колонок
    column_names = list(filter(None, lines[header_index].strip().split('\t')))

    # Извлекаем данные после заголовка
    data_lines = lines[header_index + 1:]

    # Создаем список для хранения данных
    data = []

    # Обрабатываем каждую строку данных
    for line in data_lines:
        # Разделяем строку по табуляции и удаляем пустые строки
        row = list(filter(None, line.strip().split('\t')))
        # Проверяем, что строка не пустая
        if row:
            # Преобразуем строки в числа (float)
            data.append([float(value) for value in row])

    # Проверяем, что данные были добавлены
    if not data:
        raise ValueError("No data found in the file after the header.")

    # Создаем DataFrame
    df = pd.DataFrame(data, columns=column_names)

    return df

# Пример использования
file_mot = '/content/drive/MyDrive/Colab Notebooks/AI_University/Стажировки/!03-Каратэ/10_weak/kinematics_01/kinematics/Karate_Cell_1_0-1161.mot'
df = read_mot_file(file_mot)

In [ ]:
df

,time,pelvis_tilt,pelvis_list,pelvis_rotation,pelvis_tx,pelvis_ty,pelvis_tz,hip_flexion_r,hip_adduction_r,hip_rotation_r,...,pro_sup_r,wrist_flex_r,wrist_dev_r,arm_flex_l,arm_add_l,arm_rot_l,elbow_flex_l,pro_sup_l,wrist_flex_l,wrist_dev_l
0,0.000000,14.479902,11.768095,-186.659308,0.457378,0.645875,-0.673857,16.684209,-2.997537,16.401158,...,36.456475,0.0,5.061303,-73.719035,36.180829,-17.613064,75.809089,73.742311,0.0,5.061303
1,0.034483,13.942246,11.387373,-186.114457,0.455540,0.646903,-0.674462,15.565153,-3.474138,17.045814,...,47.778830,0.0,5.044661,-74.035205,36.237272,-17.007892,75.673834,74.487112,0.0,5.044661
2,0.068966,13.733843,11.666130,-185.648842,0.453079,0.648286,-0.673263,14.721912,-3.360191,17.362090,...,76.460271,0.0,5.013748,-73.809656,36.172940,-16.870036,74.844784,75.514231,0.0,5.013748
3,0.103448,13.939310,12.094952,-184.323339,0.456156,0.650385,-0.675469,15.131791,-2.902332,15.532044,...,75.487285,0.0,5.011172,-73.319952,34.650428,-16.010693,75.239411,74.348428,0.0,5.011172
4,0.137931,13.602412,11.892827,-185.043269,0.455346,0.650476,-0.675624,14.985479,-2.834081,15.913850,...,69.072876,0.0,5.009485,-73.550291,35.616502,-17.486073,74.991274,74.293412,0.0,5.009485
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1156,39.862069,155.020887,134.933276,119.152764,0.706300,0.655662,-0.556171,58.082835,3.884276,-48.722248,...,0.628623,0.0,5.000000,1257.896903,-1227.872620,3464.372840,81.916376,75.137112,0.0,5.000000
1157,39.896552,167.580504,158.612505,102.152070,0.714400,0.671962,-0.581047,36.957049,7.241050,-16.254700,...,76.427108,0.0,5.000000,1239.610611,-1244.343795,3425.142970,75.022464,77.282235,0.0,5.000000
1158,39.931034,136.365068,107.955239,113.698145,0.804141,0.613667,-0.071848,43.047668,15.479038,-6.008087,...,0.854454,0.0,5.000000,1258.843563,-1254.438440,3576.591842,0.814178,2.136365,0.0,5.000000
1159,39.965517,138.994189,108.861033,111.359299,0.783669,0.634669,-0.105171,45.963475,17.749278,-5.033554,...,1.933335,0.0,5.000000,1252.482147,-1258.135831,3536.585687,0.651090,2.088834,0.0,5.000000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1161 entries, 0 to 1160
Data columns (total 63 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   time                   1161 non-null   float64
 1   pelvis_tilt            1161 non-null   float64
 2   pelvis_list            1161 non-null   float64
 3   pelvis_rotation        1161 non-null   float64
 4   pelvis_tx              1161 non-null   float64
 5   pelvis_ty              1161 non-null   float64
 6   pelvis_tz              1161 non-null   float64
 7   hip_flexion_r          1161 non-null   float64
 8   hip_adduction_r        1161 non-null   float64
 9   hip_rotation_r         1161 non-null   float64
 10  knee_angle_r           1161 non-null   float64
 11  knee_angle_r_beta      1161 non-null   float64
 12  ankle_angle_r          1161 non-null   float64
 13  subtalar_angle_r       1161 non-null   float64
 14  mtp_angle_r            1161 non-null   float64
 15  hip_

In [ ]:
'''
63 параметра кинематики
1. **time**: Время, в течение которого были собраны данные (обычно в секундах).
2. **pelvis_tilt**: Наклон таза вперед или назад (в градусах).
3. **pelvis_list**: Боковой наклон таза (в градусах).
4. **pelvis_rotation**: Вращение таза (в градусах).
5. **pelvis_tx**: Смещение таза по оси X (в метрах).
6. **pelvis_ty**: Смещение таза по оси Y (в метрах).
7. **pelvis_tz**: Смещение таза по оси Z (в метрах).
8. **hip_flexion_r**: Сгибание правого бедра (в градусах).
9. **hip_adduction_r**: Приведение правого бедра (в градусах).
10. **hip_rotation_r**: Вращение правого бедра (в градусах).
11. **knee_angle_r**: Угол в правом колене (в градусах).
12. **knee_angle_r_beta**: Дополнительный параметр, возможно, указывающий на другую характеристику коленного сустава (в градусах).
13. **ankle_angle_r**: Угол в правом голеностопном суставе (в градусах).
14. **subtalar_angle_r**: Угол в правом подтаранном суставе (в градусах).
15. **mtp_angle_r**: Угол в правом плюсневом суставе (в градусах).
16. **hip_flexion_l**: Сгибание левого бедра (в градусах).
17. **hip_adduction_l**: Приведение левого бедра (в градусах).
18. **hip_rotation_l**: Вращение левого бедра (в градусах).
19. **knee_angle_l**: Угол в левом колене (в градусах).
20. **knee_angle_l_beta**: Дополнительный параметр для левого колена (в градусах).
21. **ankle_angle_l**: Угол в левом голеностопном суставе (в градусах).
22. **subtalar_angle_l**: Угол в левом подтаранном суставе (в градусах).
23. **mtp_angle_l**: Угол в левом плюсневом суставе (в градусах).
24. **L5_S1_Flex_Ext**: Сгибание/разгибание в сегменте L5-S1 (в градусах).
25. **L5_S1_Lat_Bending**: Боковое сгибание в сегменте L5-S1 (в градусах).
26. **L5_S1_axial_rotation**: Осевое вращение в сегменте L5-S1 (в градусах).
27. **L4_L5_Flex_Ext**: Сгибание/разгибание в сегменте L4-L5 (в градусах).
28. **L4_L5_Lat_Bending**: Боковое сгибание в сегменте L4-L5 (в градусах).
29. **L4_L5_axial_rotation**: Осевое вращение в сегменте L4-L5 (в градусах).
30. **L3_L4_Flex_Ext**: Сгибание/разгибание в сегменте L3-L4 (в градусах).
31. **L3_L4_Lat_Bending**: Боковое сгибание в сегменте L3-L4 (в градусах).
32. **L3_L4_axial_rotation**: Осевое вращение в сегменте L3-L4 (в градусах).
33. **L2_L3_Flex_Ext**: Сгибание/разгибание в сегменте L2-L3 (в градусах).
34. **L2_L3_Lat_Bending**: Боковое сгибание в сегменте L2-L3 (в градусах).
35. **L2_L3_axial_rotation**: Осевое вращение в сегменте L2-L3 (в градусах).
36. **L1_L2_Flex_Ext**: Сгибание/разгибание в сегменте L1-L2 (в градусах).
37. **L1_L2_Lat_Bending**: Боковое сгибание в сегменте L1-L2 (в градусах).
38. **L1_L2_axial_rotation**: Осевое вращение в сегменте L1-L2 (в градусах).
39. **L1_T12_Flex_Ext**: Сгибание/разгибание в сегменте L1-T12 (в градусах).
40. **L1_T12_Lat_Bending**: Боковое сгибание в сегменте L1-T12 (в градусах).
41. **L1_T12_axial_rotation**: Осевое вращение в сегменте L1-T12 (в градусах).
42. **Abs_r3**: Абсолютное положение правой руки (возможно, в градусах или метрах).
43. **Abs_r2**: Абсолютное положение правой руки (возможно, в градусах или метрах).
44. **Abs_r1**: Абсолютное положение правой руки (возможно, в градусах или метрах).
45. **Abs_t1**: Абсолютное положение торса (возможно, в градусах или метрах).
46. **Abs_t2**: Абсолютное положение торса (возможно, в градусах или метрах).
47. **neck_flexion**: Сгибание шеи (в градусах).
48. **neck_bending**: Боковое сгибание шеи (в градусах).
49. **neck_rotation**: Вращение шеи (в градусах).
50. **arm_flex_r**: Сгибание правой руки (в градусах).
51. **arm_add_r**: Приведение правой руки (в градусах).
52. **arm_rot_r**: Вращение правой руки (в градусах).
53. **elbow_flex_r**: Сгибание правого локтя (в градусах).
54. **pro_sup_r**: Пронация/супинация правой руки (в градусах).
55. **wrist_flex_r**: Сгибание правого запястья (в градусах).
56. **wrist_dev_r**: Боковое отклонение правого запястья (в градусах).
57. **arm_flex_l**: Сгибание левой руки (в градусах).
58. **arm_add_l**: Приведение левой руки (в градусах).
59. **arm_rot_l**: Вращение левой руки (в градусах).
60. **elbow_flex_l**: Сгибание левого локтя (в градусах).
61. **pro_sup_l**: Пронация/супинация левой руки (в градусах).
62. **wrist_flex_l**: Сгибание левого запястья (в градусах).
63. **wrist_dev_l**: Боковое отклонение левого запястья (в градусах).
'''

In [ ]:
#Добавление номера кадра
import pandas as pd

# Предположим, что ваш DataFrame называется df
df['frame'] = (df['time'] * 30).astype(int)

In [ ]:
df

,time,pelvis_tilt,pelvis_list,pelvis_rotation,pelvis_tx,pelvis_ty,pelvis_tz,hip_flexion_r,hip_adduction_r,hip_rotation_r,...,wrist_flex_r,wrist_dev_r,arm_flex_l,arm_add_l,arm_rot_l,elbow_flex_l,pro_sup_l,wrist_flex_l,wrist_dev_l,frame
0,0.000000,14.479902,11.768095,-186.659308,0.457378,0.645875,-0.673857,16.684209,-2.997537,16.401158,...,0.0,5.061303,-73.719035,36.180829,-17.613064,75.809089,73.742311,0.0,5.061303,0
1,0.034483,13.942246,11.387373,-186.114457,0.455540,0.646903,-0.674462,15.565153,-3.474138,17.045814,...,0.0,5.044661,-74.035205,36.237272,-17.007892,75.673834,74.487112,0.0,5.044661,1
2,0.068966,13.733843,11.666130,-185.648842,0.453079,0.648286,-0.673263,14.721912,-3.360191,17.362090,...,0.0,5.013748,-73.809656,36.172940,-16.870036,74.844784,75.514231,0.0,5.013748,2
3,0.103448,13.939310,12.094952,-184.323339,0.456156,0.650385,-0.675469,15.131791,-2.902332,15.532044,...,0.0,5.011172,-73.319952,34.650428,-16.010693,75.239411,74.348428,0.0,5.011172,3
4,0.137931,13.602412,11.892827,-185.043269,0.455346,0.650476,-0.675624,14.985479,-2.834081,15.913850,...,0.0,5.009485,-73.550291,35.616502,-17.486073,74.991274,74.293412,0.0,5.009485,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1156,39.862069,155.020887,134.933276,119.152764,0.706300,0.655662,-0.556171,58.082835,3.884276,-48.722248,...,0.0,5.000000,1257.896903,-1227.872620,3464.372840,81.916376,75.137112,0.0,5.000000,1195
1157,39.896552,167.580504,158.612505,102.152070,0.714400,0.671962,-0.581047,36.957049,7.241050,-16.254700,...,0.0,5.000000,1239.610611,-1244.343795,3425.142970,75.022464,77.282235,0.0,5.000000,1196
1158,39.931034,136.365068,107.955239,113.698145,0.804141,0.613667,-0.071848,43.047668,15.479038,-6.008087,...,0.0,5.000000,1258.843563,-1254.438440,3576.591842,0.814178,2.136365,0.0,5.000000,1197
1159,39.965517,138.994189,108.861033,111.359299,0.783669,0.634669,-0.105171,45.963475,17.749278,-5.033554,...,0.0,5.000000,1252.482147,-1258.135831,3536.585687,0.651090,2.088834,0.0,5.000000,1198


In [ ]:
import pandas as pd

# Загрузка данных из Excel
excel_file_path = '/content/drive/MyDrive/Colab Notebooks/AI_University/Стажировки/!03-Каратэ/10_weak/10_weak-cell-01.xlsx'
cam1_df = pd.read_excel(excel_file_path, sheet_name='cam1_df')


# Проход по каждой строке cam1_df и создание частей
for index, row in cam1_df.iterrows():
    start = row['start']
    end = row['end']
    label = row['timelinelabels']

    # Фильтрация данных по фреймам
    part_df = df[(df['frame'] >= start) & (df['frame'] <= end)]

    # Сохранение в CSV файл
    filename = f"{start}_{label}.csv"
    part_df.to_csv(filename, index=False)

print("Файлы успешно сохранены.")

Файлы успешно сохранены.


In [ ]:
# Архивируем все файлы CSV в /content/
!zip csv_files.zip /content/*.csv

# Удаляем все файлы CSV в /content/
!rm /content/*.csv

  adding: content/1004_L-Turn-135.csv (deflated 53%)
  adding: content/1049_Kamae.csv (deflated 53%)
  adding: content/1072_Rei.csv (deflated 53%)
  adding: content/159_Kamae.csv (deflated 54%)
  adding: content/191_L-Turn-90.csv (deflated 54%)
  adding: content/192_L-Jodan-Haiwan-Uke.csv (deflated 53%)
  adding: content/206_L-Kokutsu-Dachi.csv (deflated 53%)
  adding: content/217_R-Chudan-Tettsui.csv (deflated 53%)
  adding: content/230_L-Chudan-Yoko-Tettsui.csv (deflated 53%)
  adding: content/253_R-Turn-180.csv (deflated 53%)
  adding: content/263_R-Jodan-Haiwan-Uke.csv (deflated 53%)
  adding: content/266_R-Kokutsu-Dachi.csv (deflated 53%)
  adding: content/280_L-Chudan-Tettsui.csv (deflated 52%)
  adding: content/292_R-Chudan-Yoko-Tettsui.csv (deflated 53%)
  adding: content/308_R-Turn-90.csv (deflated 53%)
  adding: content/315_L-Koshi-Kamae.csv (deflated 52%)
  adding: content/328_R-Jodan-Uraken.csv (deflated 53%)
  adding: content/328_R-Jodan-Yoko-Keage.csv (deflated 53%)
  add

In [ ]:
#Разархивация

import zipfile
import os

file_path = '/content/drive/MyDrive/Colab Notebooks/AI_University/Стажировки/!03-Каратэ/10_weak/kinematics_01/mot_csv/csv_files.zip'

# Папка, куда вы хотите разархивировать файлы
extract_folder = '/content/drive/MyDrive/Colab Notebooks/AI_University/Стажировки/!03-Каратэ/10_weak/kinematics_01/mot_csv'

# Создаем папку, если она не существует
os.makedirs(extract_folder, exist_ok=True)

# Разархивируем файл
with zipfile.ZipFile(file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

print(f'Файлы разархивированы в {extract_folder}')

Файлы разархивированы в /content/drive/MyDrive/Colab Notebooks/AI_University/Стажировки/!03-Каратэ/10_weak/kinematics_01/mot_csv


##Файлы кинематики участника 1 с разбивкой на Lable

https://drive.google.com/drive/folders/1Qx3_NlHPBmCzYjcn6Sso-exbpC67ly6V?usp=sharing

In [ ]:
#@title Создание клипа с 4-мя распознанными видео - участник 2

'''
Synchronizing...
--> Camera cam01 and cam02: 79 frames offset, correlation 0.64.
--> Camera cam01 and cam03: 39 frames offset, correlation 0.44.
--> Camera cam01 and cam04: -87 frames offset, correlation 0.28.
'''
import subprocess
import json
import cv2 as cv
from moviepy.editor import VideoFileClip, clips_array

# Исходные видео
video_output_1 = '/content/drive/My Drive/video/cell_1/cam01_pose.mp4'
video_output_2 = '/content/drive/My Drive/video/cell_1/cam02_pose.mp4'
video_output_3 = '/content/drive/My Drive/video/cell_1/cam03_pose.mp4'
video_output_4 = '/content/drive/My Drive/video/cell_1/cam04_pose.mp4'

output_video_list = [video_output_1, video_output_2, video_output_3, video_output_4]

# Получение FPS для каждого видео
fps_list = []
for video_path in output_video_list:
    cap = cv.VideoCapture(video_path)
    fps = cap.get(cv.CAP_PROP_FPS)
    print(fps)
    fps_list.append(fps)
    cap.release()

# Загрузка видео
video_clips = [VideoFileClip(video) for video in output_video_list]

# Синхронизация видео
# cam01 (основное видео)
cam01_duration = video_clips[0].duration

# cam04: опережает на 87 кадров
cam04_offset = 87 / fps_list[3]
video_clips[3] = video_clips[3].subclip(cam04_offset, cam01_duration)

# cam03: отстает на 39 кадров
cam03_offset = 39 / fps_list[2]
video_clips[2] = video_clips[2].subclip(cam03_offset, cam01_duration)

# cam02: отстает на 79 кадров
cam02_offset = 79 / fps_list[1]
video_clips[1] = video_clips[1].subclip(cam02_offset, cam01_duration)

# Создание плитки из видео
final_video = clips_array([[video_clips[0], video_clips[1]],
                            [video_clips[2], video_clips[3]]])

# Сохранение и вывод финального видео
final_video_path = '/content/drive/My Drive/video/cell_1/final_video_02_3.mp4'
final_video.write_videofile(final_video_path, codec='libx264')

29.8
30.005
30.002
30.0
Moviepy - Building video /content/drive/My Drive/video/cell_1/final_video_02_3.mp4.
Moviepy - Writing video /content/drive/My Drive/video/cell_1/final_video_02_3.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/My Drive/video/cell_1/final_video_02_3.mp4


##Распознанные позы участника 2

https://drive.google.com/file/d/1Qx3MFAWpfb6c5m3XU3EEfit2fbSnnLS-/view?usp=sharing

# Файл кинематики участника 2

In [ ]:
import zipfile
import os

file_path = '/content/drive/MyDrive/Colab Notebooks/AI_University/Стажировки/!03-Каратэ/10_weak/kinematics_02.zip'

# Папка, куда вы хотите разархивировать файлы
extract_folder = '/content/drive/MyDrive/Colab Notebooks/AI_University/Стажировки/!03-Каратэ/10_weak/kinematics_02/'

# Создаем папку, если она не существует
os.makedirs(extract_folder, exist_ok=True)

# Разархивируем файл
with zipfile.ZipFile(file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

print(f'Файлы разархивированы в {extract_folder}')

Файлы разархивированы в /content/drive/MyDrive/Colab Notebooks/AI_University/Стажировки/!03-Каратэ/10_weak/kinematics_02/
